In [1]:
import os
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model, ensemble, tree
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score,GridSearchCV,cross_validate
from sklearn.metrics import confusion_matrix, auc,roc_auc_score,roc_curve,recall_score,classification_report
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100, 'display.max_columns', 400)


import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, auc,roc_auc_score,roc_curve,recall_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import joblib
from mlxtend.plotting import plot_confusion_matrix

import warnings
warnings.filterwarnings("ignore")


import theano
import theano.tensor as Tensor
import random
import sys
import csv
import time
import utils
import visualize
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, BatchNormalization, Bidirectional, LayerNormalization
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import regularizers
import re
from keras.layers import Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import Activation, Dense, Flatten 

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
data4 = pd.read_csv('data4.csv')

In [3]:
data4 = data4.drop(['Unnamed: 0'], axis = 1)

In [4]:
feature = data4.drop(['Result'], axis = 1)
target  = data4['Result'].values

train,test= train_test_split(data4, test_size=0.3,random_state=123, stratify=data4.Result)# stratify the outcome
X_train =train.drop(['Result'], axis = 1)
X_test = test.drop(['Result'], axis = 1)
Y_train = train['Result'].values
Y_test = test['Result'].values

In [14]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Tensorflow/Keras: 2.9.0


In [15]:
##### Step 1 - Specify parameters
timestep=18
scaler = MinMaxScaler(feature_range=(-1, 1))

In [16]:
# Use fit to train the scaler on the training data only, actual scaling will be done inside reshaping function
scaler.fit(data4.to_numpy().reshape(-1, 1))

MinMaxScaler(feature_range=(-1, 1))

In [17]:
feature = data4.drop(['Result'], axis = 1)
target  = data4['Result'].values

train,test= train_test_split(data4, test_size=0.3,random_state=123, stratify=data4.Result)# stratify the outcome
X_train =train.drop(['Result'], axis = 1)
X_test = test.drop(['Result'], axis = 1)
Y_train = train['Result'].values
Y_test = test['Result'].values

In [19]:
X_test = np.expand_dims(X_test, axis=2)

In [20]:
X_test.shape

(395863, 63, 1)

In [21]:
X_train = np.expand_dims(X_train, axis=2)

In [22]:
X_train.shape

(923679, 63, 1)

In [26]:
model = Sequential()
# GRU LAYER IS ADDED TO MODEL WITH 128 CELLS IN IT
model.add(GRU(128, input_shape=(X_train.shape[1],X_train.shape[2]), activation='tanh', return_sequences=True))
model.add(Dropout(0.2))  # 20% DROPOUT ADDED FOR REGULARIZATION
model.add(BatchNormalization())
model.add(GRU(128, input_shape=(X_train.shape[1],X_train.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(GRU(128, input_shape=(X_train.shape[1],X_train.shape[2]), activation='tanh', return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))  # ADD A DENSE LAYER
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))  # FINAL CLASSIFICATION LAYER WITH 2 CLASSES AND SOFTMAX
# ---------------------------------------------------------------------------------------------------
# OPTIMIZER SETTINGS
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

# MODEL COMPILE
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, 63, 128)           50304     
                                                                 
 dropout (Dropout)           (None, 63, 128)           0         
                                                                 
 batch_normalization_1 (Batc  (None, 63, 128)          512       
 hNormalization)                                                 
                                                                 
 gru_2 (GRU)                 (None, 63, 128)           99072     
                                                                 
 dropout_1 (Dropout)         (None, 63, 128)           0         
                                                                 
 batch_normalization_2 (Batc  (None, 63, 128)          512       
 hNormalization)                                      

In [ ]:
history=model.fit(
    X_train, Y_train, validation_data=(X_test, Y_test), batch_size=64, epochs=10
)

Epoch 1/10
14433/14433 [==============================] - 1699s 117ms/step - loss: 0.4168 - accuracy: 0.8195 - val_loss: 0.2919 - val_accuracy: 0.8756
Epoch 2/10
14433/14433 [==============================] - 2038s 141ms/step - loss: 0.2367 - accuracy: 0.8996 - val_loss: 0.1860 - val_accuracy: 0.9224
Epoch 3/10
14433/14433 [==============================] - 1696s 118ms/step - loss: 0.2015 - accuracy: 0.9139 - val_loss: 0.1380 - val_accuracy: 0.9375
Epoch 4/10
 1502/14433 [==>...........................] - ETA: 30:23 - loss: 0.1598 - accuracy: 0.9294

In [23]:
model = keras.Sequential()
model.add(layers.GRU(64, input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 64)                12864     
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 10)                650       
                                                                 
Total params: 13,770
Trainable params: 13,642
Non-trainable params: 128
_________________________________________________________________
None


In [24]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)

In [25]:
history=model.fit(
    X_train, Y_train, validation_data=(X_test, Y_test), batch_size=64, epochs=10
)

Epoch 1/10
14433/14433 [==============================] - 395s 27ms/step - loss: 0.5064 - accuracy: 0.7921 - val_loss: 0.5104 - val_accuracy: 0.8046
Epoch 2/10
14433/14433 [==============================] - 383s 27ms/step - loss: 0.4675 - accuracy: 0.8041 - val_loss: 0.8586 - val_accuracy: 0.3574
Epoch 3/10
14433/14433 [==============================] - 387s 27ms/step - loss: 0.4600 - accuracy: 0.8047 - val_loss: 0.8323 - val_accuracy: 0.3724
Epoch 4/10
14433/14433 [==============================] - 384s 27ms/step - loss: 0.4569 - accuracy: 0.8056 - val_loss: 0.4568 - val_accuracy: 0.8070
Epoch 5/10
14433/14433 [==============================] - 386s 27ms/step - loss: 0.4558 - accuracy: 0.8058 - val_loss: 0.5643 - val_accuracy: 0.7222
Epoch 6/10
14433/14433 [==============================] - 378s 26ms/step - loss: 0.4551 - accuracy: 0.8059 - val_loss: 0.5029 - val_accuracy: 0.8042
Epoch 7/10
14433/14433 [==============================] - 388s 27ms/step - loss: 0.4538 - accuracy: 0.8061

In [32]:
##### Step 3 - Specify the structure of a Neural Network
model = Sequential(name="GRU-Model") # Model
model.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(Bidirectional(GRU(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False), name='Hidden-GRU-Encoder-Layer')) # Encoder Layer
model.add(RepeatVector(X_train.shap e[2], name='Repeat-Vector-Layer')) # Repeat Vector
model.add(Bidirectional(GRU(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False, return_sequences=True), name='Hidden-GRU-Decoder-Layer')) # Decoder Layer

In [33]:
model.summary()

Model: "GRU-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden-GRU-Encoder-Layer (B  (None, 64)               6720      
 idirectional)                                                   
                                                                 
 Repeat-Vector-Layer (Repeat  (None, 1, 64)            0         
 Vector)                                                         
                                                                 
 Hidden-GRU-Decoder-Layer (B  (None, 1, 64)            18816     
 idirectional)                                                   
                                                                 
Total params: 25,536
Trainable params: 25,536
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='sparse_categorical_crossentropy', metrics=['accuracy'],
              #loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              #metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
             )


In [ ]:
#### Step 5 - Fit the model on the dataset
history = model.fit(X_train, # input data
                    Y_train, # target data
                    batch_size=1, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs=1, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose=1, # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
                    validation_split=0.3, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 
                    validation_data=(X_test, Y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch. 
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
                    sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
                    initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
                    steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined. 
                    validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
                    validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
                    validation_freq=10, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
                    max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
                    workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
                    use_multiprocessing=True, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False. 
                   )

269148/923679 [=======>......................] - ETA: 2:39:47 - loss: 4.0534 - accuracy: 0.0366